# Train a Quantized MLP on UNSW-NB15 with Brevitas

<font color="red">**Live FINN tutorial:** We recommend clicking **Cell -> Run All** when you start reading this notebook for "latency hiding".</font>

In this notebook, we will show how to create, train and export a quantized Multi Layer Perceptron (MLP) with quantized weights and activations with [Brevitas](https://github.com/Xilinx/brevitas).
Specifically, the task at hand will be to label network packets as normal or suspicious (e.g. originating from an attacker, virus, malware or otherwise) by training on a quantized variant of the UNSW-NB15 dataset. 

**You won't need a GPU to train the neural net.** This MLP will be small enough to train on a modern x86 CPU, so no GPU is required to follow this tutorial  Alternatively, we provide pre-trained parameters for the MLP if you want to skip the training entirely.


## A quick introduction to the task and the dataset

*The task:* The goal of [*network intrusion detection*](https://ieeexplore.ieee.org/abstract/document/283931) is to identify, preferably in real time, unauthorized use, misuse, and abuse of computer systems by both system insiders and external penetrators. This may be achieved by a mix of techniques, and machine-learning (ML) based techniques are increasing in popularity. 

*The dataset:* Several datasets are available for use in ML-based methods for intrusion detection.
The **UNSW-NB15** is one such dataset created by the Australian Centre for Cyber Security (ACCS) to provide a comprehensive network based data set which can reflect modern network traffic scenarios. You can find more details about the dataset on [its homepage](https://www.unsw.adfa.edu.au/unsw-canberra-cyber/cybersecurity/ADFA-NB15-Datasets/).

*Performance considerations:* FPGAs are commonly used for implementing high-performance packet processing systems that still provide a degree of programmability. To avoid introducing bottlenecks on the network, the DNN implementation must be capable of detecting malicious ones at line rate, which can be millions of packets per second, and is expected to increase further as next-generation networking solutions provide increased
throughput. This is a good reason to consider FPGA acceleration for this particular use-case.

## Outline
-------------

* [Load the UNSW_NB15 Dataset](#load_dataset) 
* [Define a PyTorch Device](#define_pytorch_device)
* [Define the Quantized MLP Model](#define_quantized_mlp)
* [Define Train and Test  Methods](#train_test)
    * [(Option 1) Train the Model from Scratch](#train_scratch)
    * [(Option 2) Load Pre-Trained Parameters](#load_pretrained)
* [Network Surgery Before Export](#network_surgery)
* [Export to QONNX and Conversion to FINN-ONNX](#export_qonnx)

In [1]:
import os
import onnx
import torch
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  # Importing Matplotlib for plotting
from PIL import Image, ImageOps
from tqdm import tqdm, trange

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models

from brevitas.nn import QuantLinear, QuantReLU, QuantConv2d
from brevitas.quant import Int8ActPerTensorFloat, Int8WeightPerTensorFloat, Int8Bias
from brevitas.quant_tensor import QuantTensor
from brevitas.export import export_qonnx

from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN

base_dir = os.environ['FINN_ROOT'] + "/notebooks/FINN_Brevitas/"
build_dir = base_dir + "build/"

if os.path.exists(build_dir):
    shutil.rmtree(build_dir)
    print("Previous run results deleted!")

os.mkdir(build_dir)

Previous run results deleted!


**This is important -- always import onnx before torch**. This is a workaround for a [known bug](https://github.com/onnx/onnx/issues/2394).

# Load the UNSW_NB15 Dataset <a id='load_dataset'></a>

### Dataset Quantization <a id='dataset_qnt'></a>

The goal of this notebook is to train a Quantized Neural Network (QNN) to be later deployed as an FPGA accelerator generated by the FINN compiler. Although we can choose a variety of different precisions for the input, [Murovic and Trost](https://ev.fe.uni-lj.si/1-2-2019/Murovic.pdf) have previously shown we can actually binarize the inputs and still get good (90%+) accuracy.

We will create a binarized representation for the dataset by following the procedure defined by Murovic and Trost, which we repeat briefly here:

* Original features have different formats ranging from integers, floating numbers to strings.
* Integers, which for example represent a packet lifetime, are binarized with as many bits as to include the maximum value. 
* Another case is with features formatted as strings (protocols), which are binarized by simply counting the number of all different strings for each feature and coding them in the appropriate number of bits.
* Floating-point numbers are reformatted into fixed-point representation.
* In the end, each sample is transformed into a 593-bit wide binary vector. 
* All vectors are labeled as bad (0) or normal (1)

Following Murovic and Trost's open-source implementation provided as a Matlab script [here](https://github.com/TadejMurovic/BNN_Deployment/blob/master/cybersecurity_dataset_unswb15.m), we've created a [Python version](dataloader_quantized.py).

<font color="red">**Live FINN tutorial:** Downloading the original dataset and quantizing it can take some time, so we provide a download link to the pre-quantized version for your convenience. </font>

In [2]:
#! wget -O unsw_nb15_binarized.npz https://zenodo.org/record/4519767/files/unsw_nb15_binarized.npz?download=1

We can extract the binarized numpy arrays from the .npz archive and wrap them as a PyTorch `TensorDataset` as follows:

In [3]:
"""

import numpy as np
from torch.utils.data import TensorDataset

def get_preqnt_dataset(data_dir: str, train: bool):
    unsw_nb15_data = np.load(data_dir + "/unsw_nb15_binarized.npz")
    if train:
        partition = "train"
    else:
        partition = "test"
    part_data = unsw_nb15_data[partition].astype(np.float32)
    part_data = torch.from_numpy(part_data)
    part_data_in = part_data[:, :-1]
    part_data_out = part_data[:, -1]
    return TensorDataset(part_data_in, part_data_out)

train_quantized_dataset = get_preqnt_dataset(".", True)
test_quantized_dataset = get_preqnt_dataset(".", False)

print("Samples in each set: train = %d, test = %s" % (len(train_quantized_dataset), len(test_quantized_dataset))) 
print("Shape of one input sample: " +  str(train_quantized_dataset[0][0].shape))

"""

data_path = base_dir + "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"

# Prepare dataset
def load_dataset(data_path):
    images = []
    labels = []
    for subfolder in os.listdir(data_path):
        subfolder_path = os.path.join(data_path, subfolder)
        if not os.path.isdir(subfolder_path):
            continue
        for image_filename in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, image_filename)
            images.append(image_path)
            labels.append(subfolder)
    return pd.DataFrame({'image': images, 'label': labels})

data = load_dataset(data_path)
train_df, dummy_df = train_test_split(data, train_size=0.8, shuffle=True, stratify=data['label'], random_state=123)
valid_df, test_df = train_test_split(dummy_df, train_size=0.5, shuffle=True, stratify=dummy_df['label'], random_state=123)

# Define Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None, class_indices=None):
        self.dataframe = dataframe
        self.transform = transform
        self.class_indices = class_indices

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image']
        image = Image.open(img_path).convert('RGB')
        #image = ImageOps.grayscale(Image.open(img_path)) 
        label = self.class_indices[self.dataframe.iloc[idx]['label']]

        if self.transform:
            image = self.transform(image)

        return image, label

input_size = (3, 224, 224)

transform = transforms.Compose([
    transforms.Resize(input_size[1:3]),
    transforms.RandomHorizontalFlip(),
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485], std=[0.229])
])

# Create datasets and loaders
class_indices = {label: idx for idx, label in enumerate(train_df['label'].unique())}
train_dataset = CustomDataset(train_df, transform=transform, class_indices=class_indices)
valid_dataset = CustomDataset(valid_df, transform=transform, class_indices=class_indices)
test_dataset = CustomDataset(test_df, transform=transform, class_indices=class_indices)

print("Training set shape:", train_df.shape)  # Printing the shape of the training set
print("Validation set shape:", valid_df.shape)  # Printing the shape of the validation set
print("Test set shape:", test_df.shape)  # Printing the shape of the test set
print(class_indices)  # Printing the class indices


Training set shape: (9956, 2)
Validation set shape: (1245, 2)
Test set shape: (1245, 2)
{'Normal': 0, 'Tumor': 1, 'Stone': 2, 'Cyst': 3}


## Set up DataLoader

Following either option, we now have access to the quantized dataset. We will wrap the dataset in a PyTorch `DataLoader` for easier access in batches.

In [4]:
batch_size = 8

# dataset loaders
#train_quantized_loader = DataLoader(train_quantized_dataset, batch_size=batch_size, shuffle=True)
#test_quantized_loader = DataLoader(test_quantized_dataset, batch_size=batch_size, shuffle=False)    

train_quantized_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_quantized_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
count = 0
for x,y in train_quantized_loader:
    print("Input shape for 1 batch: " + str(x.shape))
    print("Label shape for 1 batch: " + str(y.shape))
    count += 1
    if count == 1:
        break

Input shape for 1 batch: torch.Size([8, 3, 224, 224])
Label shape for 1 batch: torch.Size([8])


In [6]:
g_dict = class_indices  # Storing class indices
classes = list(g_dict.keys())  # Getting class names
images, labels = next(iter(train_quantized_loader))  # Getting a batch of images and labels

plt.figure(figsize=(12, 12))  # Setting the figure size

for i in range(batch_size):  # Iterating through the first 16 images
    plt.subplot(4, 4, i + 1)  # Creating a subplot
    image = images[i].permute(1, 2, 0).numpy()  # Permuting and converting the image to numpy
    #image = images[i].numpy() 
    image = np.clip(image, 0, 1)  # Clipping the image values to [0, 1]
    plt.imshow(image)  # Displaying the image
    index = labels[i].item()  # Getting the label index
    class_name = classes[index]  # Getting the class name
    plt.title(class_name, color='blue', fontsize=10)  # Setting the title of the subplot
    plt.axis('off')  # Hiding the axis
plt.show()  # Displaying the figure

# Define a PyTorch Device <a id='define_pytorch_device'></a> 

GPUs can significantly speed-up training of deep neural networks. We check for availability of a GPU and if so define it as target device.

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Target device: " + str(device))

Target device: cuda


# Define the Quantized MLP Model <a id='define_quantized_mlp'></a>

We'll now define an MLP model that will be trained to perform inference with quantized weights and activations.
For this, we'll use the quantization-aware training (QAT) capabilities offered by [Brevitas](https://github.com/Xilinx/brevitas).

Our MLP will have four fully-connected (FC) layers in total: three hidden layers with 64 neurons, and a final output layer with a single output, all using 2-bit weights. We'll use 2-bit quantized ReLU activation functions, and apply batch normalization between each FC layer and its activation.

In case you'd like to experiment with different quantization settings or topology parameters, we'll define all these topology settings as variables.

In [8]:
#input_size = 593      
#hidden1 = 64      
#hidden2 = 64
#hidden3 = 64
#weight_bit_width = 2
#act_bit_width = 2
#num_classes = 1    

Now we can define our MLP using the layer primitives provided by Brevitas:

In [9]:
# Setting seeds for reproducibility
torch.manual_seed(0)

"""
model = nn.Sequential(
      QuantLinear(input_size[0]*input_size[1]*input_size[2], 64, bias=True, weight_bit_width=8),
      nn.BatchNorm1d(64),
      nn.Dropout(0.5),
      QuantReLU(bit_width=8),
      QuantLinear(64, 32, bias=True, weight_bit_width=8),
      nn.BatchNorm1d(32),
      nn.Dropout(0.5),
      QuantReLU(bit_width=8),
      QuantLinear(32, 16, bias=True, weight_bit_width=8),
      nn.BatchNorm1d(16),
      nn.Dropout(0.5),
      QuantReLU(bit_width=8),
      QuantLinear(16, 4, bias=True, weight_bit_width=8)
)

model.to(device)
"""

# Define custom MobileNetV2 model
class MobileNetV2Model(nn.Module):  # Defining a custom MobileNetV2 model class
    def __init__(self, num_classes=4):  # Initializing the model
        super(MobileNetV2Model, self).__init__()  # Calling the superclass constructor
        self.base_model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)  # Loading a pretrained MobileNetV2 model
        self.base_model.classifier[1] = nn.Sequential(  # Modifying the classifier layer
            nn.Linear(self.base_model.last_channel, 128),  # Adding a Linear layer
            nn.ReLU(),  # Adding a ReLU activation function
            nn.Dropout(0.5),  # Adding a Dropout layer
            nn.Linear(128, num_classes),  # Adding a Linear layer for the output
            nn.Softmax(dim=1)  # Adding a Softmax activation function
        )

    def forward(self, x):  # Defining the forward pass
        return self.base_model(x)  # Returning the output of the base model
        
class QuantMobileNetV2Model(nn.Module):
    def __init__(self, num_classes=4):
        super(QuantMobileNetV2Model, self).__init__()
        self.base_model = models.mobilenet_v2(pretrained=True)
        self.base_model.features[0][0] = QuantConv2d(1, 32, 3, bias=True, weight_bit_width=8)
        self.base_model.features[0][1] = QuantReLU(bit_width=8)
        
        # Replace all Conv2d and ReLU layers with quantized versions
        for i, layer in enumerate(self.base_model.features):
            if isinstance(layer, nn.Conv2d):
                self.base_model.features[i] = qnn.QuantConv2d.from_float(layer, weight_bit_width=8, bias_quant=Int8Bias, weight_quant=Int8WeightPerTensorFloat)
            elif isinstance(layer, nn.ReLU):
                self.base_model.features[i] = qnn.QuantReLU(quant_type=QuantType.INT, bit_width=8)
                
        self.base_model.classifier[1] = nn.Sequential(
            nn.Linear(self.base_model.last_channel, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
        
    def forward(self, x):
        return self.base_model(x)


class TestModel1(nn.Module):
    def __init__(self, num_classes=4):
        super(TestModel1, self).__init__()

        self.conv1 = QuantConv2d(3, 6, 5, bias=True, weight_bit_width=8, padding=2)
        self.lin2_2 = QuantLinear(int(6*input_size[1]*input_size[2]/4), 32, bias=True, weight_bit_width=8)
        
        self.lin1 = QuantLinear(input_size[0]*input_size[1]*input_size[2], 64, bias=True, weight_bit_width=8)
        self.bnorm1 = nn.BatchNorm1d(64)
        self.drop1 = nn.Dropout(0.5)
        self.relu1 = QuantReLU(bit_width=8)
        self.lin2 = QuantLinear(64, 32, bias=True, weight_bit_width=8)
        self.bnorm2 = nn.BatchNorm1d(32)
        self.drop2 = nn.Dropout(0.5)
        self.relu2 = QuantReLU(bit_width=8)
        self.lin3 = QuantLinear(32, 16, bias=True, weight_bit_width=8)
        self.bnorm3 = nn.BatchNorm1d(16)
        self.drop3 = nn.Dropout(0.5)
        self.relu3 = QuantReLU(bit_width=8)
        self.lin4 = QuantLinear(16, 4, bias=True, weight_bit_width=8)

    def forward(self, x):
        
        out = x.view(-1, input_size[0]*input_size[1]*input_size[2])
        out = self.lin1(out)
        out = self.bnorm1(out)
        out = self.drop1(out)
        out = self.relu1(out)
        out = self.lin2(out)

        #out = self.conv1(x)
        #out = self.relu1(out)
        #out = nn.functional.max_pool2d(out, 2)
        #out = out.view(-1, int(6*input_size[1]*input_size[2]/4))
        #out = self.lin2_2(out)
         
        out = self.bnorm2(out)
        #print(out.shape)
        out = self.drop2(out)
        #print(out.shape)
        out = self.relu2(out)
        out = self.lin3(out)
        out = self.bnorm3(out)
        #print(out.shape)
        out = self.drop3(out)
        #print(out.shape)
        out = self.relu3(out)
        out = self.lin4(out)
        #print(out.shape)

        return out
     
# Define the Simple CNN model inside an nn.Sequential block
class SimpleSequentialCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(SimpleSequentialCNN, self).__init__()
        
        # Define the convolutional layers and pooling layers
        self.model = nn.Sequential(
            # First convolutional block
            QuantConv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding=1, weight_bit_width=8),
            nn.BatchNorm2d(2),
            QuantReLU(bit_width=8),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 32 x 112 x 112
            
            # Second convolutional block
            QuantConv2d(in_channels=2, out_channels=4, kernel_size=3, stride=1, padding=1, weight_bit_width=8),
            nn.BatchNorm2d(4),
            QuantReLU(bit_width=8),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 64 x 56 x 56
            
            # Third convolutional block
            QuantConv2d(in_channels=4, out_channels=8, kernel_size=3, stride=1, padding=1, weight_bit_width=8),
            nn.BatchNorm2d(8),
            QuantReLU(bit_width=8),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 128 x 28 x 28
            
            # Flatten the tensor
            nn.Flatten(),  # Output: 128 * 28 * 28 = 100352
            
            # Fully connected layer
            nn.Linear(8 * 28 * 28, num_classes)
            #nn.Linear(128 * 8 * 8, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Instantiate the model with the fixed input size of (3, 224, 224)
model = SimpleSequentialCNN(num_classes=4).to(device)


Note that the MLP's output is not yet quantized. Even though we want the final output of our MLP to be a binary (0/1) value indicating the classification, we've only defined a single-neuron FC layer as the output. While training the network we'll pass that output through a sigmoid function as part of the loss criterion, which [gives better numerical stability](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html). Later on, after we're done training the network, we'll add a quantization node at the end before we export it to FINN.

# Define Train and Test  Methods  <a id='train_test'></a>
The train and test methods will use a `DataLoader`, which feeds the model with a new predefined batch of training data in each iteration, until the entire training data is fed to the model. Each repetition of this process is called an `epoch`.

In [10]:
def train(model, train_loader, optimizer, criterion):
    losses = []
    # ensure model is in training mode
    model.train()    
    
    for i, data in enumerate(train_loader, 0):        
        inputs, target = data
        inputs, target = inputs.to(device), target.to(device)
        if inputs.shape[0] != batch_size:
            continue
        optimizer.zero_grad()   
        quant_images = QuantTensor(inputs, scale=1.0, bit_width=8)        
        # forward pass
        output = model(quant_images)
        #loss = criterion(output, target.unsqueeze(1))
        loss = criterion(output, target)
        
        # backward pass + run optimizer to update weights
        loss.backward()
        optimizer.step()
        
        # keep track of loss value
        losses.append(loss.data.cpu().numpy()) 
           
    return losses

In [11]:
def test(model, test_loader):    
    # ensure model is in eval mode
    model.eval() 
    y_true = []
    y_pred = []
   
    with torch.no_grad():
        for data in test_loader:
            inputs, target = data
            inputs, target = inputs.to(device), target.to(device)
            if inputs.shape[0] != batch_size:
                continue
            quant_images = QuantTensor(inputs, scale=1.0, bit_width=8)
            output_orig = model(quant_images)
            # run the output through sigmoid
            #output = torch.sigmoid(output_orig)  
            # compare against a threshold of 0.5 to generate 0/1
            #pred = (output.detach().cpu().numpy() > 0.5) * 1
            _, pred = torch.max(output_orig, 1)
            target = target.cpu().float()
            y_true.extend(target.tolist()) 
            y_pred.extend(pred.reshape(-1).tolist())
        
    return accuracy_score(y_true, y_pred)

# Train the QNN <a id="train_qnn"></a>

We provide two options for training below: you can opt for training the model from scratch (slower) or use a pre-trained model (faster). The first option will give more insight into how the training process works, while the second option will likely give better accuracy.

## (Option 1, slower) Train the Model from Scratch <a id="train_scratch"></a>


Before we start training our MLP we need to define some hyperparameters. Moreover, in order to monitor the loss function evolution over epochs, we need to define a method for it. As mentioned earlier, we'll use a loss criterion which applies a sigmoid function during the training phase (`BCEWithLogitsLoss`). For the testing phase, we're manually computing the sigmoid and thresholding at 0.5.

In [12]:
num_epochs = 3
lr = 0.001 

def display_loss_plot(losses, title="Training loss", xlabel="Iterations", ylabel="Loss"):
    x_axis = [i for i in range(len(losses))]
    plt.plot(x_axis,losses)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

In [13]:
# loss criterion and optimizer
#criterion = nn.BCEWithLogitsLoss().to(device)
#optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

# Define loss and optimizer
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, inputs, targets):
        #print(inputs.shape)
        #print(targets.shape)
        BCE_loss = nn.functional.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        return F_loss.mean()

criterion = FocalLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
# Setting seeds for reproducibility
torch.manual_seed(0)
np.random.seed(0)

running_loss = []
running_test_acc = []
t = trange(num_epochs, desc="Training loss", leave=True)

for epoch in t:
        loss_epoch = train(model, train_quantized_loader, optimizer,criterion)
        test_acc = test(model, test_quantized_loader)
        t.set_description("Training loss = %f test accuracy = %f" % (np.mean(loss_epoch), test_acc))
        t.refresh() # to show immediately the update           
        running_loss.append(loss_epoch)
        running_test_acc.append(test_acc)

Training loss:   0%|                                      | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:1255: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at ../c10/core/TensorImpl.h:1758.)
  return super(Tensor, self).rename(names)
Training loss = 0.008360 test accuracy = 0.989516:  67%|▋| 2/3 [01:55<00:57, 57.

In [ ]:
%matplotlib inline

loss_per_epoch = [np.mean(loss_per_epoch) for loss_per_epoch in running_loss]
display_loss_plot(loss_per_epoch)

In [ ]:
acc_per_epoch = [np.mean(acc_per_epoch) for acc_per_epoch in running_test_acc]
display_loss_plot(acc_per_epoch, title="Test accuracy", ylabel="Accuracy [%]")

In [ ]:
test(model, test_quantized_loader)

In [ ]:
# Save the Brevitas model to disk
torch.save(model.state_dict(), build_dir + "state_dict_self-trained.pth")

## (Option 2, faster) Load Pre-Trained Parameters <a id="load_pretrained"></a>

Instead of training from scratch, you can also use pre-trained parameters we provide here. These parameters should achieve ~91.9% test accuracy.

In [ ]:
#import os
#import torch

# Make sure the model is on CPU before loading a pretrained state_dict
#model = model.cpu()

# Load pretrained weights
#trained_state_dict = torch.load(model_dir + "/state_dict.pth")["models_state_dict"][0]

#model.load_state_dict(trained_state_dict, strict=False)

In [ ]:
# Move the model back to it's target device
#model.to(device)

# Test for accuracy
#test(model, test_quantized_loader)

**Why do these parameters give better accuracy vs training from scratch?** Even with the topology and quantization fixed, achieving good accuracy on a given dataset requires [*hyperparameter tuning*](https://towardsdatascience.com/hyperparameters-optimization-526348bb8e2d) and potentially running training for a long time. The "training from scratch" example above is only intended as a quick example, whereas the pretrained parameters are obtained from a longer training run using the [determined.ai](https://determined.ai/) platform for hyperparameter tuning.

# Network Surgery Before Export <a id="network_surgery"></a>

Sometimes, it's desirable to make some changes to our trained network prior to export (this is known in general as "network surgery"). This depends on the model and is not generally necessary, but in this case we want to make a couple of changes to get better results with FINN.

In [ ]:
# Move the model to CPU before surgery
model = model.cpu()

Let's start by padding the input. Our input vectors are 593-bit, which will make folding (parallelization) for the first layer a bit tricky since 593 is a prime number. So we'll pad the weight matrix of the first layer with seven 0-valued columns to work with an input size of 600 instead. When using the modified network we'll similarly provide inputs padded to 600 bits.

In [ ]:
#from copy import deepcopy

#modified_model = deepcopy(model)

#W_orig = modified_model[0].weight.data.detach().numpy()
#W_orig.shape

In [ ]:
#import numpy as np

# pad the second (593-sized) dimensions with 7 zeroes at the end
#W_new = np.pad(W_orig, [(0,0), (0,7)])
#W_new.shape

In [ ]:
#modified_model[0].weight.data = torch.from_numpy(W_new)
#modified_model[0].weight.shape

Next, we'll modify the expected input/output ranges. In FINN, we prefer to work with bipolar {-1, +1} instead of binary {0, 1} values. To achieve this, we'll create a "wrapper" model that handles the pre/postprocessing as follows:

* on the input side, we'll pre-process by (x + 1) / 2 in order to map incoming {-1, +1} inputs to {0, 1} ones which the trained network is used to. Since we're just multiplying/adding a scalar, these operations can be [*streamlined*](https://finn.readthedocs.io/en/latest/nw_prep.html#streamlining-transformations) by FINN and implemented with no extra cost.

* on the output side, we'll add a binary quantizer which maps everthing below 0 to -1 and everything above 0 to +1. This is essentially the same behavior as the sigmoid we used earlier, except the outputs are bipolar instead of binary.

In [ ]:
#from brevitas.nn import QuantIdentity


#class CybSecMLPForExport(nn.Module):
#    def __init__(self, my_pretrained_model):
#        super(CybSecMLPForExport, self).__init__()
#        self.pretrained = my_pretrained_model
#        self.qnt_output = QuantIdentity(
#            quant_type='binary', 
#            scaling_impl_type='const',
#            bit_width=1, min_val=-1.0, max_val=1.0)
    
#    def forward(self, x):
#        # assume x contains bipolar {-1,1} elems
#        # shift from {-1,1} -> {0,1} since that is the
#        # input range for the trained network
#        x = (x + torch.tensor([1.0]).to(x.device)) / 2.0  
#        out_original = self.pretrained(x)
#        out_final = self.qnt_output(out_original)   # output as {-1,1}     
#        return out_final

#model_for_export = CybSecMLPForExport(modified_model)
#model_for_export.to(device)

model_for_export = model
model_for_export.to(device)

In [ ]:
"""

def test_padded_bipolar(model, test_loader):    
    # ensure model is in eval mode
    model.eval() 
    y_true = []
    y_pred = []
   
    with torch.no_grad():
        for data in test_loader:
            inputs, target = data
            inputs, target = inputs.to(device), target.to(device)
            # pad inputs to 600 elements
            input_padded = torch.nn.functional.pad(inputs, (0,7,0,0))
            # convert inputs to {-1,+1}
            input_scaled = 2 * input_padded - 1
            # run the model
            output = model(input_scaled.float())
            y_pred.extend(list(output.flatten().cpu().numpy()))
            # make targets bipolar {-1,+1}
            expected = 2 * target.float() - 1
            expected = expected.cpu().numpy()
            y_true.extend(list(expected.flatten()))
        
    return accuracy_score(y_true, y_pred)
"""

In [ ]:
#test_padded_bipolar(model_for_export, test_quantized_loader)

# Export to QONNX and Conversion to FINN-ONNX <a id="export_qonnx" ></a>


[ONNX](https://onnx.ai/) is an open format built to represent machine learning models, and the FINN compiler expects an ONNX model as input. We'll now export our network into ONNX to be imported and used in FINN for the next notebooks. Note that the particular ONNX representation used for FINN differs from standard ONNX, you can read more about this [here](https://finn.readthedocs.io/en/latest/internals.html#intermediate-representation-finn-onnx).

You can see below how we export a trained network in Brevitas into a FINN-compatible ONNX representation (QONNX). QONNX is the format we can export from Brevitas, to feed it into the FINN compiler, we will need to make a conversion to the FINN-ONNX format which is the intermediate representation the compiler works on. The conversion of the FINN-ONNX format is a FINN compiler transformation and to be able to apply it to our model, we will need to wrap it into [ModelWrapper](https://finn.readthedocs.io/en/latest/internals.html#modelwrapper). This is a wrapper around the ONNX model which provides several helper functions to make it easier to work with the model. Then we can call the conversion function to obtain the model in FINN-ONNX format.

In [ ]:
brevitas_model_filename = build_dir + "brevitas-ready-2.onnx"
ready_model_filename = build_dir + "/finn-brevitas-ready.onnx"
input_shape = (1, input_size[0], input_size[1], input_size[2])

print(input_shape)

# create a QuantTensor instance to mark input as bipolar during export
input_a = np.random.randint(0, 1.0, size=input_shape).astype(np.float32)
#input_a = 2 * input_a - 1
scale = 1.0
input_t = torch.from_numpy(input_a * scale)
#input_t = QuantTensor(input_t, scale=1.0, zero_point=0.0, bit_width=8, signed=False, training=True)
#Move to CPU before export
model_for_export.cpu()

# Export to ONNX
export_qonnx(model_for_export, export_path=brevitas_model_filename, input_t=input_t)

# clean-up
qonnx_cleanup(brevitas_model_filename, out_file=brevitas_model_filename)

# ModelWrapper
model = ModelWrapper(brevitas_model_filename)
# Setting the input datatype explicitly because it doesn't get derived from the export function
#model.set_tensor_datatype(model.graph.input[0].name, DataType["FLOAT16"])
model = model.transform(ConvertQONNXtoFINN())
model.save(ready_model_filename)

print("Model saved to %s" % ready_model_filename)

## View the Exported ONNX in Netron

Let's examine the exported ONNX model with [Netron](https://github.com/lutzroeder/netron), which is a visualizer for neural networks and allows interactive investigation of network properties. For example, you can click on the individual nodes and view the properties. Particular things of note:

* The input tensor "0" is annotated with `quantization: finn_datatype: BIPOLAR`
* The input preprocessing (x + 1) / 2 is exported as part of the network (initial `Add` and `Div` layers)
* Brevitas `QuantLinear` layers are exported to ONNX as `MatMul`. We've exported the padded version; shape of the first MatMul node's weight parameter is 600x64
* The weight parameters (second inputs) for MatMul nodes are annotated with `quantization: finn_datatype: INT2`
* The quantized activations are exported as `MultiThreshold` nodes with `domain=qonnx.custom_op.general`
* There's a final `MultiThreshold` node with threshold=0 to produce the final bipolar output (this is the `qnt_output` from `CybSecMLPForExport`

In [ ]:
from finn.util.visualization import showInNetron

showInNetron(ready_model_filename)

## That's it! <a id="thats_it" ></a>
You created, trained and tested a quantized MLP that is ready to be loaded into FINN, congratulations! You can now proceed to the next notebook.